In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import pandas as pd
import numpy as np
import json
import torch
import re
import ast

In [ ]:
model_name = "Qwen/Qwen2.5-14B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="mps"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [ ]:
system_prompt = "You are an expert at solving complex grade-school math problems with step-by-step reason and return output in json format"

user_prompt_template = """{question}

Solve the above grade-school math problem by breaking it down step by step.

Also, present only numeric value for Answer. Follow this format strictly. No any other format:
"Reason:" [step 1: ..., step 2: ..., ...]
"Answer:" [final numeric answer] 
"""


In [ ]:
main_path = '../gsm8k/similarity_generate_questions/'

In [ ]:
files = os.listdir(main_path)
if '.DS_Store' in files:
    files.remove('.DS_Store')
len(files)

In [ ]:
with open(main_path + files[0], 'r') as f:
    data = json.load(f)

In [ ]:
data

In [ ]:
data['generated_question'][-1]

In [ ]:
   
def save_pred(question, reason, answer, GT_answer, GT_reason, generated_question, evaluation, question_num, ques_similarity_value, response, path):
    response = response.replace("'", "")
    try:
        reason_start = re.search(r"Reason:", response, re.DOTALL).span()[0]
        reason_end = re.search(r"Reason:", response, re.DOTALL).span()[1]
        answer_start = re.search(r"Answer:", response, re.DOTALL).span()[0]
        answer_end = re.search(r"Answer:", response, re.DOTALL).span()[1]
    except:
        reason_start = re.search(r'"Reason":', response, re.DOTALL).span()[0]
        reason_end = re.search(r'"Reason":', response, re.DOTALL).span()[1]
        answer_start = re.search(r'"Answer":', response, re.DOTALL).span()[0]
        answer_end = re.search(r'"Answer":', response, re.DOTALL).span()[1]

    reason_text = response[reason_start:answer_start]
    reason_lines = [line.strip() for line in reason_text.strip().split("\n") if line.strip()]
    
    answer_text = response[answer_start:]
    answer_str = answer_text.replace("Answer:", "").strip()
    try:
        answer_dict = ast.literal_eval(answer_str)
    except:
        answer_dict = answer_str.strip()
    
    result = {
            "question": question,
            "reason":reason,
            "answer":answer,
            "GT_answer":GT_answer,
            "GT_reason":GT_reason,
            "generated_question":generated_question,
            "evaluation":evaluation,
            "question_num":question_num,
            "ques_similarity_value":ques_similarity_value,

            "pred_reason": reason_lines,
            "pred_answer": answer_dict,
        }
    with open(path, "w") as f:
        json.dump(result, f, indent=2)



In [ ]:


for num, file_name in enumerate(files):
    if num < 395:
        continue
    print(num)
    with open(main_path + file_name, 'r') as f:
        data = json.load(f)
    
    question = data['question']
    reason = data['reason']
    answer = data['answer']
    GT_answer = data['GT_answer']
    GT_reason = data['GT_reason']
    generated_question = data['generated_question']
    evaluation = data['evaluation']
    question_num = data['question_num']
    ques_similarity_value = data['ques_similarity_value']
        
    num = question_num[-1]
    if num == 2:
        ques = generated_question[-1][22:-1]
    elif num == 0:
        ques = generated_question[0][14:-2]
    elif num == 1:
        ques = generated_question[1][14:-2]
    
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_template.format(question=ques)}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    path = '../gsm8k/answer_reason_generated_question/' + file_name
    save_pred(question, reason, answer, GT_answer, GT_reason, generated_question, evaluation, question_num, ques_similarity_value, response, path)
   
       

In [ ]:
response